# Geodetic Frame,ECEF Frame, and NED Frame

It's hard to compute distances using lat and lon. So it's best we use another frame. Convert to a local ECEF. Build an ECEF frame around your drone take-off point. Z is height. XY is lateral distance. XY is tangent to surface of Earth. X usually points north. Y usually points east. Z points down. First define a gobal home for NED origin. Then convert an NED translation from NED origin into Geodetic (lat, long, alt). 

1. Given global position, global home, return local position. 

2. Given local position, global home, return global position. 

Local position therefore means NED relative to origin. Global position means lat, long, alt. 

In [2]:
import sys
!{sys.executable} -m pip install utm

Learn to use utm.from_latlon
-> Converts lat lon to easting, northing within a zone number, letter.
Learn to use utm.to_latlon

In [8]:
import utm
import numpy
(easting, northing, zone_number, zone_letter) = utm.from_latlon(0,0)
print(easting, northing, zone_number, zone_letter)
(latitude, longitude) = utm.to_latlon(easting, northing, zone_number, zone_letter)
print(latitude,longitude)

166021.4431793313 0.0 31 N
1.008118449665103e-07 1.2752752050796116e-10


Implement global_to_local() and local_to_global()
From global geodetic to UTM and vv.


In [31]:
import utm
import numpy as np

def global_to_local(global_position, global_home):
    """
    Inputs: global_position: numpy array [longitude, latitude, altitude]; global_home: numpy array: [longitude, latitude, 
        altitude]
    Returns: local_position numpy array [northing, easting, altitude] of global_position
        relative to global_home [longitude, latitude, altitude]; Essentially, returns a
        numpy array describing the NED delta from global_home.
    """
    # Get easting and northing of global home first
    lonh, lath, alth = global_home[0], global_home[1], global_home[2]
    (easting_h, northing_h, zone_number_h, zone_letter_h) = utm.from_latlon(lath, lonh)
    
    # Get easting and northing from global position next
    lon, lat, alt = global_position[0], global_position[1], global_position[2]
    (easting, northing, zone_number, zone_letter) = utm.from_latlon(lat, lon)
    
    # After that, create a local_position numpy array from its NED coordinates
    local_position = np.array([northing - northing_h, easting - easting_h, -(alt - alth)])
    
    # Finally, return them.
    return local_position

def local_to_global(local_position, global_home):
    """
    Inputs: local_position: numpy array [northing, easting, down]; global_home: numpy array [longitude, latitude, altitude]
    Returns: global_position: numpy array [longitude, latitude, altitude]. Essentially, returns a numpy array holding a new
        geodetic location given a home geodetic location and an NED delta. 
    """
    # First, get global_home's easting, northing, zone_number, and grid_letter.
    lon_h, lat_h, alt_h = global_home[0], global_home[1], global_home[2]
    (easting_h, northing_h, zone_number_h, zone_letter_h) = utm.from_latlon(lat_h, lon_h)
    
    # After that, compute the new NED location by adding local_position to home's NED coordinates.
    northing, easting, alt = local_position[0], local_position[1], local_position[2] 
    
    # Convert the new NED location to the geodetic frame next. 
    (latitude, longitude) = utm.to_latlon(easting + easting_h, northing + northing_h, zone_number_h, zone_letter_h)
    altitude = -(alt - alt_h)
    global_position = np.array([longitude, latitude, altitude])
    
    # Finally, return the new geodetic location.
    return global_position


In [34]:
np.set_printoptions(precision=2)

geodetic_current_coordinates = [-122.079465, 37.393037, 30]
geodetic_home_coordinates = [-122.108432, 37.400154, 20]

local_coordinates_NED = global_to_local(geodetic_current_coordinates, geodetic_home_coordinates)

print(local_coordinates_NED)
# Should print [ -764.96  2571.59   -10.  ]

[-764.96 2571.59  -10.  ]


In [35]:
np.set_printoptions(precision=6)
NED_coordinates =[25.21, 128.07, -30.]

# convert back to global coordinates
geodetic_current_coordinates = local_to_global(NED_coordinates, geodetic_home_coordinates)

print(geodetic_current_coordinates)
# Should print [-122.106982   37.40037    50.      ]

[-122.106982   37.40037    50.      ]
